In [1]:
using Plots
using DataFrames
using CSV
using Query

backend()

Plots.GRBackend()

In [2]:
x = 1:10; y = rand(10,2) # 2 columns means two lines
plot(x,y,title="abc")
savefig("myplot.pdf") # Saves the plot from p as a .pdf vector graphic

In [3]:
CSV.validate("output/output-without-elements.csv", delim=";", header=1)

data = CSV.read("output/output-without-elements.csv", delim=";", header=1)
head(data)

,Tool,View,ChangeSet,RunIndex,Iteration,PhaseName,MetricName,MetricValue
1,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Initialization,Time,183510119
2,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Initialization,Memory,7913400
3,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Load,Time,285321797
4,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Load,Memory,7428792
5,jastadd-ttc18-relast-emf-inc,Q2,1,0,0,Initialization,Time,176043764
6,jastadd-ttc18-relast-emf-inc,Q2,1,0,0,Initialization,Memory,7913448


In [4]:
data1 = @from i in data begin
        @where i.MetricName == "Time"
        @let value = i.MetricValue / 10^6
        @select {i.Tool, i.View, i.ChangeSet, i.PhaseName, i.MetricValue, Value=value}
        @collect DataFrame
end
data1[:ShortTool] = map((x) -> x[15:end], data1[:Tool])
head(data1)

,Tool,View,ChangeSet,PhaseName,MetricValue,Value,ShortTool
1,jastadd-ttc18-relast-emf-inc,Q1,1,Initialization,183510119,183.51,relast-emf-inc
2,jastadd-ttc18-relast-emf-inc,Q1,1,Load,285321797,285.322,relast-emf-inc
3,jastadd-ttc18-relast-emf-inc,Q2,1,Initialization,176043764,176.044,relast-emf-inc
4,jastadd-ttc18-relast-emf-inc,Q2,1,Load,246596896,246.597,relast-emf-inc
5,jastadd-ttc18-relast-emf-inc,Q1,2,Initialization,179378789,179.379,relast-emf-inc
6,jastadd-ttc18-relast-emf-inc,Q1,2,Load,334370109,334.37,relast-emf-inc


In [5]:
datag = by(data1, [:ShortTool, :ChangeSet, :View, :PhaseName], df -> mean(df[:Value]))
rename!(datag, :x1 => :Mean)
head(datag)

,ShortTool,ChangeSet,View,PhaseName,Mean
1,relast-emf-inc,1,Q1,Initialization,183.51
2,relast-emf-inc,1,Q1,Load,285.322
3,relast-emf-inc,1,Q2,Initialization,176.044
4,relast-emf-inc,1,Q2,Load,246.597
5,relast-emf-inc,2,Q1,Initialization,179.379
6,relast-emf-inc,2,Q1,Load,334.37


In [6]:
minY = floor(log10(minimum(datag[:Mean])))
maxY = log10(maximum(datag[:Mean]))
println(minY, " ", maxY)

-1.0 3.6943785108614433


In [7]:
phase_names = Dict("Load" => "01-Load", "Initial" => "02-Initial", "Update" => "03-Update")

good_colors = ["#000000","#e69f00","#56b4e9","#009e73","#f0e442","#0072b2","#d55e00","#cc79a7"]
# possible: [:none,:auto,:circle,:rect,:star5,:diamond,:hexagon,:cross,:xcross,:utriangle,:dtriangle,
#            :rtriangle,:ltriangle,:pentagon,:heptagon,:octagon,:star4,:star6,:star7,:star8,:vline,:hline,:+,:x]
good_shapes = [:circle, :rect, :utriangle, :star4]
# possible dashes are [:auto,:solid,:dash,:dot,:dashdot,:dashdotdot]
good_dashes = [:dot, :solid, :dash, :dashdot]

struct DiagramInfo
    color::String
    shape::Symbol
    dash::Symbol
    name::String
end

# color, shape, dash-type, name in legend
diagram_info = Dict(
    "xml-inc" =>          DiagramInfo(good_colors[1], good_shapes[1], good_dashes[1], "Incremental"),
    "xml-flush" =>        DiagramInfo(good_colors[2], good_shapes[2], good_dashes[2], "Flushing"),
    "relast-xml-inc" =>   DiagramInfo(good_colors[3], good_shapes[3], good_dashes[3], "Incremental (Relations)"),
    "relast-xml-flush" => DiagramInfo(good_colors[4], good_shapes[4], good_dashes[4], "Flushing (Relations)"),
    "relast-emf-inc" =>   DiagramInfo(good_colors[5], good_shapes[1], good_dashes[1], "Incremental (EMF)"),
    "relast-emf-flush" => DiagramInfo(good_colors[8], good_shapes[2], good_dashes[2], "Flushing (EMF)")
)

Dict{String,DiagramInfo} with 6 entries:
  "relast-emf-flush" => DiagramInfo("#cc79a7", :rect, :solid, "Flushing (EMF)")
  "xml-inc"          => DiagramInfo("#000000", :circle, :dot, "Incremental")
  "xml-flush"        => DiagramInfo("#e69f00", :rect, :solid, "Flushing")
  "relast-xml-flush" => DiagramInfo("#009e73", :star4, :dashdot, "Flushing (Rel…
  "relast-xml-inc"   => DiagramInfo("#56b4e9", :utriangle, :dash, "Incremental …
  "relast-emf-inc"   => DiagramInfo("#f0e442", :circle, :dot, "Incremental (EMF…

In [38]:
#viewToPlot = "Q1"
#phaseToPlot = "Load"
function draw_scatter(datag, viewToPlot, phaseToPlot; drawLegend=false)
    xtickvals = [1,2,4,8,16,32,64]
    ytickvals = [minor * 10 ^ major for major = 0 : ceil(maxY) for minor = 1 : 10]
    yticklabels = [minor == 1 ? trunc(Int, 10 ^ major) : "" for major = 0 : ceil(maxY) for minor = 1 : 10]
    f = font("Computer Modern", 16)
    t = plot(scale = :log, foreground_color_border="black", border=true,
        xlims = (0.9, maximum(datag[:ChangeSet]) + 5), xticks = (xtickvals,xtickvals), xguide = "ChangeSet",
        ylims = (1, 10000), yticks = (ytickvals, yticklabels), yguide = "Time (ms)",
        foreground_color_grid="black", grid = !drawLegend, tickfont=f, guidefont=f)

    for toolname = map((x) -> lowercase(x), unique(datag[:ShortTool]))
        sub = @from i in datag begin
              @where i.View == viewToPlot && i.PhaseName == phaseToPlot && i.ShortTool == toolname
              @select {i.ChangeSet, i.Mean}
              @collect DataFrame
        end
        info = diagram_info[toolname]
        plot!(sub[:ChangeSet], sub[:Mean], label=info.name,
              linecolor=info.color, linestyle=info.dash,
              markershape=info.shape, markercolor=info.color,
              legend = drawLegend ? :bottom : :none)
    end
    phaseName = phase_names[phaseToPlot]
    filename = "diagrams/plot-$viewToPlot-$phaseName"
    if drawLegend
        filename = filename * "-Legend"
    end
    savefig(filename * ".pdf")
end

draw_scatter (generic function with 1 method)

In [39]:
needLegend = true
for query = (printAll ? ["Q1", "Q2"] : ["Q1"])
    for phase = (printAll ? ["Load", "Initial", "Update"] : ["Update"])
        #new_data = output.query('View == @query & PhaseName == @phase')
        draw_it = (draw_legend -> draw_scatter(datag, query, phase, drawLegend=needLegend))
        if needLegend
            draw_it(true)
            needLegend = false
        end
        draw_it(false)
    end
end